In [1]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models
import os
import matplotlib.pyplot as plt
from torch.optim import Adam
from tqdm import tqdm_notebook

print(torch.__version__)
device = torch.device('cuda:1')

1.2.0


In [2]:
class Weedread(Dataset):
    def __init__(self, name, transform=None):
        hf = h5py.File(name, 'r')
        input_images = np.array(hf.get('data'), np.uint8)
        target_labels = np.array(hf.get('labels')).astype(np.long)
        
        self.input_images = input_images
        self.target_labels = target_labels
        self.transform = transform
        hf.close()

    def __len__(self):
        return self.input_images.shape[0]

    def __getitem__(self, idx):
        images = self.input_images[idx]
        classes = self.target_labels[idx][1]
        family =  self.target_labels[idx][0]
        if self.transform is not None:
            images = self.transform(images)
        images = images
        
        return images, classes, family

In [3]:
INPUT_CHANNEL = 3
BATCH_SIZE = 32
normalize = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((96,96)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

imagenet_classes = range(1, 22)

data_path = os.path.dirname(os.getcwd()) + "/data/weed/"
Train_data = Weedread(data_path + "train.h5", transform=normalize)
Test_data = Weedread(data_path + "val.h5", transform=normalize)

Train_dataloader = DataLoader(dataset=Train_data,
                              batch_size = BATCH_SIZE,
                              shuffle=True)
Test_dataloader = DataLoader(dataset=Test_data,
                              batch_size = BATCH_SIZE,
                              shuffle=False)


In [4]:

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class My_Model(nn.Module):
    def __init__(self, input_channel=1, num_class=21, num_family = 5):
        super(My_Model, self).__init__()
        model = torchvision.models.mobilenet_v2(pretrained=True)
        self.model_ft = torch.nn.Sequential(*(list(model.features.children())[:19]))
        set_parameter_requires_grad(self.model_ft, False)

        self.family_fc = nn.Linear(20480, num_family)
        self.class_fc = nn.Linear(20480, num_class)
        
    
    def forward(self, x):
        # Perform the usual forward pass
        x = self.model_ft(x)
        x = torch.flatten(x, 1)
        x_class = self.class_fc(x)
        x_family = self.family_fc(x)
        return F.softmax(x_class, dim=1), F.softmax(x_family, dim=1)

In [5]:
likelihood_matrix = torch.randn(5, 21).fill_(10e-6).to(device)
likelihood_matrix[0, 0:11] = 1
likelihood_matrix[1, 11] = 1
likelihood_matrix[2, 12:14] = 1
likelihood_matrix[3, 14:19] = 1
likelihood_matrix[4, 19:21] = 1

class _loss(nn.Module):
    def __init__(self, alpha=0.5):
        super(_loss, self).__init__()
        self.alpha = alpha
        self.class_loss = nn.CrossEntropyLoss()
        self.family_loss = nn.CrossEntropyLoss()
    def forward(self, predicted_class, true_class, predicted_family, true_family):
        return self.alpha * self.class_loss(predicted_class, true_class) + \
                (1-self.alpha)*self.family_loss(predicted_family, true_family)

In [6]:
from torchsummary import summary
train_images, _, _ = next(iter(Test_dataloader))


_model = My_Model(num_class = 21, num_family = 5)
summary(_model, input_size= train_images[0].size(), device="cpu")
_model = _model.to(device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
             ReLU6-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]             288
       BatchNorm2d-5           [-1, 32, 64, 64]              64
             ReLU6-6           [-1, 32, 64, 64]               0
            Conv2d-7           [-1, 16, 64, 64]             512
       BatchNorm2d-8           [-1, 16, 64, 64]              32
  InvertedResidual-9           [-1, 16, 64, 64]               0
           Conv2d-10           [-1, 96, 64, 64]           1,536
      BatchNorm2d-11           [-1, 96, 64, 64]             192
            ReLU6-12           [-1, 96, 64, 64]               0
           Conv2d-13           [-1, 96, 32, 32]             864
      BatchNorm2d-14           [-1, 96,

In [7]:
from tqdm import tqdm_notebook

optimizer = torch.optim.SGD(_model.parameters(), lr=0.01)
criterion = _loss(alpha = 1)
EPOCHS = 100

max_correct = 0

for epoch in range(1, EPOCHS + 1):
    #training
    _model.train()
    for image, classes, family in tqdm_notebook(Train_dataloader):
        image, classes, family = image.to(device), classes.to(device), family.to(device)
        image = image.float()
        p_classes, p_family = _model(image)
        loss = criterion(p_classes, classes, p_family, family)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, EPOCHS))
    #testing
    _model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for image, classes, family in (Test_dataloader):
            image, classes, family = image.to(device), classes.to(device), family.to(device)
            image = image.float()
            p_classes, p_family = _model(image)
            predicted = torch.argmax(p_classes,dim=1)
            total += image.size(0)
            correct += (predicted == classes).sum().item()
        print('Test Accuracy of the model on the test images: {:.4f} %'.format(100 * correct / total))
    if(correct > max_correct):
        max_correct = correct
        torch.save(_model.state_dict(), 'epochs/Mobile-baseline-model.pt')


Loss :2.3908 Epoch[1/100]
Test Accuracy of the model on the test images: 65.9002 %



Loss :2.3921 Epoch[2/100]
Test Accuracy of the model on the test images: 67.2865 %



Loss :2.5343 Epoch[3/100]
Test Accuracy of the model on the test images: 67.9702 %



Loss :2.4295 Epoch[4/100]
Test Accuracy of the model on the test images: 68.3287 %



Loss :2.4298 Epoch[5/100]
Test Accuracy of the model on the test images: 68.6227 %


...


Loss :2.2774 Epoch[33/100]
Test Accuracy of the model on the test images: 83.1437 %



Loss :2.3143 Epoch[34/100]
Test Accuracy of the model on the test images: 83.1437 %



Loss :2.1995 Epoch[35/100]
Test Accuracy of the model on the test images: 83.2823 %



Loss :2.1999 Epoch[36/100]
Test Accuracy of the model on the test images: 84.3580 %


KeyboardInterrupt: 